**Install python dependencies**

In [ ]:
!pip install -q -r ./requirements.txt

**Load python libraries**

In [103]:
from sqlalchemy import create_engine
from os import getenv
import pandas as pd
import dask.dataframe as dd
from datetime import datetime
import smart_match

pd.options.mode.chained_assignment = None  # default='warn'

**Connect to database**

In [ ]:
db_name = getenv("POSTGRES_DB")
db_user = getenv("POSTGRES_USER")
db_pass = getenv("POSTGRES_PASSWORD")
db_host = 'localhost'
db_port = '5432'
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

**Preprocess Data: Event Data**

In [111]:
# getting event data from database
db_response = db.execute(
    f"""select  (coalesce(p.first_name, '') || ' ' ||coalesce(p.last_name, '')) as name,
       position,
       club_id,
       number as matchday,
       type as event_type,
       count(*) from events
        inner join matchdays m on events.matchday_id = m.id
        inner join players p on events.player_id = p.id
    where corrected is false
    group by name, position, matchday, event_type, club_id
    order by name, matchday,  event_type
    """
)

df_event = pd.DataFrame(db_response.fetchall())
df_event.columns = db_response.keys()

df_event

,name,position,club_id,matchday,event_type,count
0,Aarón,defender,e6c49a1f-35e4-5c77-b845-c71eb4dea7ce,3,interception,1
1,Aarón,defender,e6c49a1f-35e4-5c77-b845-c71eb4dea7ce,3,pass,10
2,Aarón,defender,e6c49a1f-35e4-5c77-b845-c71eb4dea7ce,3,playerOn,1
3,Aarón,defender,e6c49a1f-35e4-5c77-b845-c71eb4dea7ce,3,successfulTackle,1
4,Aarón,defender,e6c49a1f-35e4-5c77-b845-c71eb4dea7ce,3,throwIn,2
...,...,...,...,...,...,...
74079,Yuya Osako,attacker,4f9c979f-4120-5d67-b1fa-b0e90aeeaba8,34,shotAtGoal,1
74080,Yuya Osako,attacker,4f9c979f-4120-5d67-b1fa-b0e90aeeaba8,34,successfulTackle,2
74081,Yuya Osako,attacker,4f9c979f-4120-5d67-b1fa-b0e90aeeaba8,34,superPass,1
74082,Yuya Osako,attacker,4f9c979f-4120-5d67-b1fa-b0e90aeeaba8,34,unsuccessfulPass,3


In [112]:
# load event_types
event_types = ['pass', 'unsuccessfulPass', 'superPass', 'throwIn', 'goalAssist', 'farCorner', 'cross', 'freeKick', 'interception', 'goalMissedFar', 'shotAtGoal', 'unsuccessfulPenalty', 'missedChance', 'goal', 'doublePack', 'hattrick', 'penaltyGoal', 'ownGoal', 'foul','awardedPenalty', 'causedPenalty', 'yellowCard', 'secondYellowCard', 'redCard', 'successfulTackle', 'unsuccessfulTackle', 'blockedGoalShot', 'offside', 'error', 'savedPenalty', 'lostPenalty', 'defended', 'goalAgainst']

In [146]:
df_players = df_event.groupby('name')

df_final = pd.DataFrame(columns=['name','position','matchday','event_type','count', 'club_id'])

# add 0 values
for player_tuple in df_players:
    player_name = player_tuple[0]
    position = player_tuple[1]['position'].max()
    club_id = player_tuple[1]['club_id'].max()

    df_player = player_tuple[1]

    for matchday in range(1,35):
        for event_type in event_types:
            if df_player.loc[(df_player['matchday'] == matchday) & (df_player['event_type'] == event_type)].empty == True:
                df_player = df_player.append({
                    'name': player_name, 
                    'position': position,
                    'club_id': club_id,
                    'matchday': matchday,
                    'event_type': event_type,
                    'count': 0
                }, ignore_index=True)
    
    df_final = df_final.append(df_player)

# to csv
df_final.to_csv('events_prep.csv', index=False)

**Preprocess Data: Betting Odds**

In [96]:
# load odds from csv
df_odds_csv = pd.read_csv('odds_20_21.csv')
df_odds = df_odds_csv[['Date', 'HomeTeam', 'AwayTeam', 'AvgH', 'AvgD', 'AvgA']]
df_odds['Date'] = df_odds['Date'].map(lambda x: datetime.strptime(datetime.strptime(x, "%d/%m/%Y").strftime("%Y-%m-%d"), "%Y-%m-%d"))
df_odds['matchday'] = 0

# add matchday to odds
df_matchdays_db = pd.read_sql_table('matchdays', db)
df_matchdays = df_matchdays_db[['start', 'end', 'number']]
df_matchdays['start'] = df_matchdays['start'].map(lambda x: datetime.strptime(x.split('T')[0], "%Y-%m-%d"))
df_matchdays['end'] = df_matchdays['end'].map(lambda x: datetime.strptime(x.split('T')[0], "%Y-%m-%d"))

for index, row in df_odds.iterrows():
    for index2, row2 in df_matchdays.iterrows():
        if row2['start'] <= row['Date'] <= row2['end']:
            df_odds.loc[index, 'matchday'] = row2['number']

,Date,HomeTeam,AwayTeam,AvgH,AvgD,AvgA,matchday
0,2020-09-18,Bayern Munich,Schalke 04,1.09,11.59,24.69,1
1,2020-09-19,Ein Frankfurt,Bielefeld,1.65,4.19,4.91,1
2,2020-09-19,FC Koln,Hoffenheim,2.60,3.60,2.59,1
3,2020-09-19,Stuttgart,Freiburg,2.26,3.48,3.16,1
4,2020-09-19,Union Berlin,Augsburg,2.34,3.25,3.19,1
...,...,...,...,...,...,...,...
301,2021-05-22,Hoffenheim,Hertha,1.94,3.99,3.60,34
302,2021-05-22,Stuttgart,Bielefeld,2.40,3.89,2.70,34
303,2021-05-22,Union Berlin,RB Leipzig,2.97,3.89,2.22,34
304,2021-05-22,Werder Bremen,M'gladbach,2.67,3.92,2.42,34


In [104]:
# game odds to win, lose, draw odds per team
df_odds_team = pd.DataFrame(columns=['team_name','matchday', 'odds_win','odds_draw','odds_lose'])

for index, row in df_odds.iterrows():
    df_odds_team = df_odds_team.append({
        'team_name': row['HomeTeam'],
        'matchday': row['matchday'],
        'odds_win': row['AvgH'],
        'odds_draw': row['AvgD'],
        'odds_lose': row['AvgA'],
    } , ignore_index=True)

    df_odds_team = df_odds_team.append({
        'team_name': row['AwayTeam'],
        'matchday': row['matchday'],
        'odds_win': row['AvgA'],
        'odds_draw': row['AvgD'],
        'odds_lose': row['AvgH'],
    } , ignore_index=True)

,team_name,matchday,odds_win,odds_draw,odds_lose
0,Bayern Munich,1,1.09,11.59,24.69
1,Schalke 04,1,24.69,11.59,1.09
2,Ein Frankfurt,1,1.65,4.19,4.91
3,Bielefeld,1,4.91,4.19,1.65
4,FC Koln,1,2.60,3.60,2.59
...,...,...,...,...,...
607,RB Leipzig,34,2.22,3.89,2.97
608,Werder Bremen,34,2.67,3.92,2.42
609,M'gladbach,34,2.42,3.92,2.67
610,Wolfsburg,34,1.66,4.30,4.81


In [110]:
# odds team names to club_ids
df_teams = pd.read_sql_table('teams', db)

def name_to_club_id(name): 
    most_similar = {'db_club_id': '', 'similarity': 0 }
    for index, db_team in df_teams.iterrows():
        similarity = smart_match.similarity(db_team['name'], name)
        if most_similar['similarity'] < similarity:
            most_similar['db_club_id'] = db_team['id']
            most<fg_similar['similarity'] = similarity
    return most_similar['db_club_id']

df_odds_team['club_id'] = df_odds_team['team_name'].map(lambda x: name_to_club_id(x))

df_odds_team.to_csv('odds_prep.csv', index=False)